In [1]:
import random
import math
import numpy as np

In [2]:
class Data:

    # initialize needed values
    def __init__(self, address):
        self.name = ""
        self.num_cities = 0
        self.locations = []
        self.address = address

    # read test case
    def load_data(self):
        with open (self.address, 'r') as f:
            self.name = f.readline().split(" ")[1]
            self.num_cities = int(self.name[2:])
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            f.readline()
            for _ in range(self.num_cities):
                city = list(map(float, f.readline().split(" ")))[1:]
                self.locations.append(city)


In [30]:
class Gene:

    # initialize parameters
    def __init__(self, num_cities, locations):
        self.gene = []
        self.num_cities = num_cities
        self.locations = locations
        self.fitness = math.inf


    # create random gene for first population
    def initial_gene(self):
        self.gene = [i for i in range(self.num_cities)]
        random.shuffle(self.gene)

    # calculate fitness
    def fitness_function(self):
        self.fitness = 0
        for i in range(self.num_cities-1):
            self.fitness += (np.sqrt((self.locations[self.gene[i]][0] - self.locations[self.gene[i+1]][0])**2 + (self.locations[self.gene[i]][1] - self.locations[self.gene[i+1]][1])**2))
        self.fitness += np.sqrt((self.locations[self.gene[-1]][0] - self.locations[self.gene[0]][0])**2 + (self.locations[self.gene[-1]][1] - self.locations[self.gene[0]][1])**2)
        return self.fitness

    # mutate gene
    def mutation(self):
        i = random.randint(1, self.num_cities-1)
        j = random.randint(1, self.num_cities-1)

        # randomly replace 2 positions in gene
        tmp = self.gene[i]
        self.gene[i] = self.gene[j]
        self.gene[j] = tmp

        self.fitness = self.fitness_function()

    # define a local search to search for neighbors of a gene that is better than itself
    def local_search(self, num_neighbors):

        neighbors = []
        for _ in range(num_neighbors):
            genei = Gene(self.num_cities, self.locations)
            genei.gene = self.gene
            i = random.randint(1, self.num_cities-1)
            j = random.randint(1, self.num_cities-1)

            # randomly change 2 positions in gene
            tmp = genei.gene[i]
            genei.gene[i] = genei.gene[j]
            genei.gene[j] = tmp

            genei.fitness = genei.fitness_function()
            neighbors.append((genei.fitness, genei))

        # select best neighbor
        neighbors.append((self.fitness, self))
        neighbors = sorted(neighbors, key=lambda x: x[0])

        return neighbors[0][1]


In [4]:
class Population:

    # initialize population's parameters
    def __init__(self, data, pop_size=100):
        self.population_size  = pop_size
        self.population       = []
        self.locations        = data.locations
        self.num_cities       = data.num_cities


    # generate first population randomly
    def initialize_population(self):
        for _ in range(self.population_size):
            genei = Gene(self.num_cities, self.locations)
            genei.initial_gene()
            self.population.append(genei)

    # generate first population starting from each city
    def initialize_population_one_city(self):
        for i in range(self.population_size):
            genei         = Gene(self.num_cities, self.locations)
            genei.initial_gene()
            genelist      = genei.gene.copy()
            ind           = genelist.index(i)
            tmp           = genelist[ind]
            genelist[ind] = genelist[0]
            genelist[0]   = tmp
            genei.gene    = genelist
            self.population.append(genei)


    # selest parents from population for crossover using tournament selection
    def parent_selection(self, N=3):
        # choose random N candidates to attend tornoment
        father_candidates = [random.choice(self.population) for _ in range(N)]
        mother_candidates = [random.choice(self.population) for _ in range(N)]

        # choose the best one of N candidates to be a parent
        # find candidates fitnesses
        father_vals = [(father.fitness, father) for father in father_candidates]
        mother_vals = [(mother.fitness, mother) for mother in mother_candidates]
        # sort candidates
        sorted_fathers = sorted(father_vals, key=lambda x: x[0])
        sorted_mothers = sorted(mother_vals, key=lambda x: x[0])
        # select parents with min fitness
        best_father = sorted_fathers[0][1]
        best_mother = sorted_mothers[0][1]

        return best_father, best_mother

     # selest parents from population for crossover using rank-based selection
    def parent_selection_rank_based(self):
        # sort population based on fitness
        fitness_values = [gene.fitness for gene in self.population]
        ranked_indices = np.argsort(fitness_values)
        ranked_population = [self.population[i] for i in ranked_indices]

        # rank calculation for selection
        selection_p = np.arange(1, self.population_size + 1) / np.sum(np.arange(1, self.population_size + 1))

        # select parents based on rank and selection probabilities
        selected_parents = np.random.choice(ranked_population, size=2, replace=False, p=selection_p)

        return selected_parents[0], selected_parents[1]


    # crossover function
    def crossover(self, father, mother):
        father = father.gene
        mother = mother.gene
        child1l, child2l = [], []
        child1, child2 = Gene(self.num_cities, self.locations), Gene(self.num_cities, self.locations)
        random_point = random.randint(1, self.num_cities-1)

        for i in range(0, random_point):
                child1l.append(father[i])
                child2l.append(mother[i])

        for i in range(random_point, self.num_cities):         # add the rest of other parent from random point if its not used before from previous parent
            if mother[i] not in child1l:
                child1l.append(mother[i])

            if father[i] not in child2l:
                child2l.append(father[i])

        for i in range(0, random_point):     # add other parent from first to the random point
            if mother[i] not in child1l:
                child1l.append(mother[i])

            if father[i] not in child2l:
                child2l.append(father[i])

        child1.gene    = child1l
        child1.fitness = child1.fitness_function()
        child2.gene    = child2l
        child2.fitness = child2.fitness_function()
        return child1, child2




### Genetic (TSP)

In [5]:
def genetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_replace, pop_size, stop_iter, expected_val, tornoment_N=3):
    # save best seen result
    best_cost = math.inf
    best_result = []

    # load test case
    testcase = Data(test_address)
    testcase.load_data()

    # step 1: create first population
    population = Population(testcase, pop_size=pop_size)
    population.initialize_population()

    iteration_without_change, g_num = 0, 0
    # iteration of running algorithm (number of generations)
    while True:
        g_num += 1 # iteration number
        children_population = Population(testcase, pop_size=pop_size)
        counter = 0
        # create as many children as parents population
        while counter < pop_size:

            # step 2: parent selection for crossover
            father1, mother1 = population.parent_selection_rank_based()

            # step 3: crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                child1, child2 = population.crossover(father1, mother1)
                counter += 2

                # step 4: mutate children with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                # add generated children to children population
                children_population.population.append(child1)
                children_population.population.append(child2)

        # step 5: population replacement
        parent_values = []
        child_values  = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness
            y = children_population.population[i].fitness
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        # replace num_replace worst parent with best children
        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))    # find the worst parent
            min_child_ind  = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # step 6: population fitness check, find the best result in population
        flag = False
        for i in range(pop_size):
            if population.population[i].fitness < best_cost:
                best_cost   = population.population[i].fitness
                best_result = population.population[i].gene.copy()
                iteration_without_change = 0
                flag = True
        if not flag:
            iteration_without_change += 1

        print(f"Iteration {g_num}, best_cost so far: {np.round(best_cost, 3)}")

        # step 7: check stop condition
        if best_cost < expected_val or iteration_without_change > stop_iter:
            break


    return best_result, testcase




Test 1

In [13]:
best_result, test = genetic_algorithm_main(test_address="gr229.tsp", generation_num=500, p_cross=0.65, p_mutate=0.6, p_replace=0.1, pop_size=100, tornoment_N=3, expected_val=2500, stop_iter=100)

Iteration 1, best_cost so far: 7229.802
Iteration 2, best_cost so far: 7229.802
Iteration 3, best_cost so far: 7229.802
Iteration 4, best_cost so far: 7229.802
Iteration 5, best_cost so far: 7229.802
Iteration 6, best_cost so far: 7229.802
Iteration 7, best_cost so far: 7204.26
Iteration 8, best_cost so far: 7204.26
Iteration 9, best_cost so far: 7204.26
Iteration 10, best_cost so far: 7204.26
Iteration 11, best_cost so far: 6986.877
Iteration 12, best_cost so far: 6986.877
Iteration 13, best_cost so far: 6986.877
Iteration 14, best_cost so far: 6986.877
Iteration 15, best_cost so far: 6932.488
Iteration 16, best_cost so far: 6932.488
Iteration 17, best_cost so far: 6803.257
Iteration 18, best_cost so far: 6803.257
Iteration 19, best_cost so far: 6803.257
Iteration 20, best_cost so far: 6803.257
Iteration 21, best_cost so far: 6803.257
Iteration 22, best_cost so far: 6803.257
Iteration 23, best_cost so far: 6803.257
Iteration 24, best_cost so far: 6803.257
Iteration 25, best_cost so fa

Test 2

In [19]:
best_result, test = genetic_algorithm_main(test_address="pr1002.tsp", generation_num=500, p_cross=0.65, p_mutate=0.6, p_replace=0.1, pop_size=100, tornoment_N=3, expected_val=1000000, stop_iter=100)

Iteration 1, best_cost so far: 1244140.715
Iteration 2, best_cost so far: 1244140.715
Iteration 3, best_cost so far: 1244087.592
Iteration 4, best_cost so far: 1244087.592
Iteration 5, best_cost so far: 1244087.592
Iteration 6, best_cost so far: 1244087.592
Iteration 7, best_cost so far: 1244087.592
Iteration 8, best_cost so far: 1239489.366
Iteration 9, best_cost so far: 1239489.366
Iteration 10, best_cost so far: 1234014.137
Iteration 11, best_cost so far: 1232028.422
Iteration 12, best_cost so far: 1232028.422
Iteration 13, best_cost so far: 1224927.973
Iteration 14, best_cost so far: 1224927.973
Iteration 15, best_cost so far: 1224927.973
Iteration 16, best_cost so far: 1222514.232
Iteration 17, best_cost so far: 1220586.078
Iteration 18, best_cost so far: 1215240.309
Iteration 19, best_cost so far: 1209846.782
Iteration 20, best_cost so far: 1209846.782
Iteration 21, best_cost so far: 1209846.782
Iteration 22, best_cost so far: 1209114.579
Iteration 23, best_cost so far: 1209114.5

### memetic (TSP)

In [22]:
def memetic_algorithm_main(test_address, generation_num, p_cross, p_mutate, p_neighbor, p_replace, pop_size, stop_iter, expected_val, tornoment_N=3):
    # save best seen result
    best_cost = math.inf
    best_result = []

    # load test case
    testcase = Data(test_address)
    testcase.load_data()

    # step 1: create first population
    population = Population(testcase, pop_size=pop_size)
    population.initialize_population()

    iteration_without_change, g_num = 0, 0
    # iteration of running algorithm (number of generations)
    while True:
        g_num += 1 # iteration number
        children_population = Population(testcase, pop_size=pop_size)
        counter = 0
        # create as many children as parents population
        while counter < pop_size:

            # step 2: parent selection for crossover
            father1, mother1 = population.parent_selection_rank_based()

            # step 3: crossover 2 selected parents with p_cross
            e_cross = random.random()
            if e_cross < p_cross:
                child1, child2 = population.crossover(father1, mother1)
                counter += 2

                # step 4: mutate children with p_mutate
                e_mutate1 = random.random()
                if e_mutate1 < p_mutate:
                    child1.mutation()
                e_mutate2 = random.random()
                if e_mutate2 < p_mutate:
                    child2.mutation()

                # step 5: local search on children and update fitness
                e_neighbor1 = random.random()
                if e_neighbor1 < p_neighbor:
                    genei = child1.local_search(num_neighbors=10)
                else:
                    genei = child1
                e_neighbor2 = random.random()
                if e_neighbor2 < p_neighbor:
                    genej = child2.local_search(num_neighbors=10)
                else:
                    genej = child2


                # add generated children to children population
                children_population.population.append(genei)
                children_population.population.append(genej)

        # step 6: population replacement
        parent_values = []
        child_values  = []
        for i in range(pop_size):             # save all values of parents and children in 2 value lists
            x = population.population[i].fitness
            y = children_population.population[i].fitness
            parent_values.append(x)
            child_values.append(y)

        num_replace = p_replace * pop_size    # find the number of parents that should be replaced with children in population

        # replace num_replace worst parent with best children
        for i in range(int(num_replace)):        # replace as the size of replacements in population
            max_parent_ind = parent_values.index(max(parent_values))    # find the worst parent
            min_child_ind  = child_values.index(min(child_values))      # find best children

            population.population.pop(max_parent_ind)      # remove worst parent from population
            population.population.append(children_population.population[min_child_ind])  # add best children to population

            children_population.population.pop(min_child_ind)  # remove best children from population of children
            parent_values.pop(max_parent_ind)   # remove worst parent value from parents values
            child_values.pop(min_child_ind)  # remove best children value from children values

        # step 7: population fitness check, find the best result in population
        flag = False
        for i in range(pop_size):
            if population.population[i].fitness < best_cost:
                best_cost   = population.population[i].fitness
                best_result = population.population[i].gene.copy()
                iteration_without_change = 0
                flag = True
        if not flag:
            iteration_without_change += 1

        print(f"Iteration {g_num}, best_cost so far: {np.round(best_cost, 3)}")

        # step 7: check stop condition
        if best_cost < expected_val or iteration_without_change > stop_iter:
            break


    return best_result, testcase




Test 1

In [27]:
best_result, test = memetic_algorithm_main(test_address="gr229.tsp", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, stop_iter=100, expected_val=2500, p_neighbor=0.5)

Iteration 1, best_cost so far: 4775.962
Iteration 2, best_cost so far: 4775.962
Iteration 3, best_cost so far: 4775.962
Iteration 4, best_cost so far: 4775.962
Iteration 5, best_cost so far: 4775.962
Iteration 6, best_cost so far: 4718.392
Iteration 7, best_cost so far: 4718.392
Iteration 8, best_cost so far: 4670.678
Iteration 9, best_cost so far: 4670.678
Iteration 10, best_cost so far: 4670.678
Iteration 11, best_cost so far: 4651.22
Iteration 12, best_cost so far: 4591.485
Iteration 13, best_cost so far: 4591.485
Iteration 14, best_cost so far: 4591.485
Iteration 15, best_cost so far: 4591.485
Iteration 16, best_cost so far: 4591.485
Iteration 17, best_cost so far: 4591.485
Iteration 18, best_cost so far: 4591.485
Iteration 19, best_cost so far: 4591.485
Iteration 20, best_cost so far: 4591.485
Iteration 21, best_cost so far: 4534.132
Iteration 22, best_cost so far: 4534.132
Iteration 23, best_cost so far: 4466.103
Iteration 24, best_cost so far: 4466.103
Iteration 25, best_cost so

Test 2

In [31]:
best_result, test = memetic_algorithm_main(test_address="pr1002.tsp", generation_num=500, p_cross=0.65, p_mutate=0.5, p_replace=0.1, pop_size=100, stop_iter=100, expected_val=1000000, p_neighbor=0.5)

Iteration 1, best_cost so far: 1259748.058
Iteration 2, best_cost so far: 1233696.99
Iteration 3, best_cost so far: 1233696.99
Iteration 4, best_cost so far: 1233696.99
Iteration 5, best_cost so far: 1233696.99
Iteration 6, best_cost so far: 1233696.99
Iteration 7, best_cost so far: 1233696.99
Iteration 8, best_cost so far: 1233696.99
Iteration 9, best_cost so far: 1225152.652
Iteration 10, best_cost so far: 1225152.652
Iteration 11, best_cost so far: 1225152.652
Iteration 12, best_cost so far: 1225152.652
Iteration 13, best_cost so far: 1225152.652
Iteration 14, best_cost so far: 1225152.652
Iteration 15, best_cost so far: 1223120.125
Iteration 16, best_cost so far: 1223120.125
Iteration 17, best_cost so far: 1222359.612
Iteration 18, best_cost so far: 1215763.456
Iteration 19, best_cost so far: 1215763.456
Iteration 20, best_cost so far: 1209823.265
Iteration 21, best_cost so far: 1209823.265
Iteration 22, best_cost so far: 1209823.265
Iteration 23, best_cost so far: 1209823.265
Iter